In [ ]:
import requests, pandas
import numpy as np
from IPython.display import Image,HTML
pandas.set_option('display.max_colwidth', -1)

In [ ]:

def reindex_from_1(df):
    df2 = df.reset_index(drop=True)
    df2.index = df2.index+1
    return df2

def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:20px;"/>'

def normalize_dict(d):
    d_max = max(d.values())
    d_min = min(d.values())
    d_range = d_max - d_min
    d_new = d
    for team in d:
        d_new[team] -= d_min
        d_new[team] /= d_range
    return d_new

def get_rank(df,team):
    return df.index[df['school'] == team].tolist()[0]

In [ ]:
game_df = pandas.read_csv("data/fbs_games_short.csv")
team_df = pandas.read_csv("data/fbs_teams_short.csv")

In [ ]:
HTML(team_df.to_html(escape=False ,formatters=dict(logo=path_to_image_html)))

In [ ]:
wins_dict = {}
losses_dict = {}
logo_dict = {}
score_dict = {}
second_order_score_dict = {}
for index,team_row in team_df.iterrows():
    team = team_row['school']
    wins_dict[team]=[]
    losses_dict[team]=[]
    logo_dict[team]=team_row['logo']
    score_dict[team] = 0
    second_order_score_dict[team] = 0
#num_wins = [len(wins_dict[x]) for x in wins_dict.keys()]

In [ ]:
#build w-l relationships
for index,game in game_df.iterrows():
    if game['home_points']>game['away_points']:
        winning_team = game['home_team']
        losing_team = game['away_team']   
    else:
        winning_team = game['away_team']
        losing_team = game['home_team']
    wins_dict[winning_team].append(losing_team)
    losses_dict[losing_team].append(winning_team)    

In [ ]:
for team in score_dict:
    print(team)
    for team_defeated in wins_dict[team]:
        score_dict[team] += (len(wins_dict[team_defeated])+1)
        print("+",team_defeated,(len(wins_dict[team_defeated])+1))
    for team_lost_to in losses_dict[team]:
        score_dict[team] -= (len(losses_dict[team_lost_to])+1)
        print("-",team_lost_to,(len(losses_dict[team_lost_to])+1))
    print("*",score_dict[team])

In [ ]:
team_df['score'] = team_df['school'].map(score_dict)
team_df = team_df.sort_values(by='score', ascending=False)
team_df = reindex_from_1(team_df)

In [ ]:
record_dict = {}
for team in wins_dict:
    num_wins = len(wins_dict[team])
    num_losses = len(losses_dict[team])
    record_dict[team] = str(num_wins) + "-" +  str(num_losses)
#score_dict = normalize_dict(score_dict)

In [ ]:
best_win_dict = {}
for team in wins_dict:
    if len(wins_dict[team]) == 0:
        best_win_dict[team] = "None"
    else:
        best_score = -99999
        for team_beat in wins_dict[team]:
            if score_dict[team_beat] > best_score:
                best_win_dict[team] = "#" + str(get_rank(team_df, team_beat)) + " " + team_beat + " (" + record_dict[team_beat] + ")"
                best_score = score_dict[team_beat]

In [ ]:
worst_loss_dict = {}
for team in losses_dict:
    if len(losses_dict[team]) == 0:
        worst_loss_dict[team] = "None"
    else:
        worst_score = 99999
        for team_lost_to in losses_dict[team]:
            if score_dict[team_lost_to] < worst_score:
                worst_loss_dict[team] = "#" + str(get_rank(team_df, team_lost_to)) + " " +team_lost_to + " (" + record_dict[team_lost_to] + ")"
                worst_score = score_dict[team_lost_to]

In [ ]:
team_df['record'] = team_df['school'].map(record_dict)
team_df['Best Win'] = team_df['school'].map(best_win_dict)
team_df['Worst Loss'] = team_df['school'].map(worst_loss_dict)
team_df = team_df[['logo','school','mascot','conference','record','score','Best Win','Worst Loss']]
HTML(team_df.to_html(escape=False ,formatters=dict(logo=path_to_image_html)))

In [ ]:
#second order calc

In [ ]:
for team in second_order_score_dict:
    for team_defeated in wins_dict[team]:
        second_order_score_dict[team] += score_dict[team_defeated]
    for team_lost_to in losses_dict[team]:
        second_order_score_dict[team] -= (1-score_dict[team_lost_to])
    print(team,second_order_score_dict[team])
second_order_score_dict = normalize_dict(second_order_score_dict)

In [ ]:

#team_df['Second Order Score'] = team_df['school'].map(second_order_score_dict)
#team_df['record'] = team_df['school'].map(record_dict)
#team_df = team_df.sort_values(by='Second Order Score', ascending=False)
#team_df = reindex_from_1(team_df)
#team_df = team_df[['logo','school','mascot','conference','record','Second Order Score']]
#HTML(team_df.to_html(escape=False ,formatters=dict(logo=path_to_image_html)))


In [ ]:
team_df.index[team_df['school'] == "Tulsa"].tolist()[0]